In [1]:
import os
import pandas as pd
# Mudança de diretório
os.chdir('E:\\Dropbox\\transito')
# Carregar o arquivo CSV
file_path = 'E:\\Dropbox\\transito\\receita\\receita_multas.csv'
data = pd.read_csv(file_path)
# Converter a coluna 'data' para o formato datetime e ordenar os dados pela data
data['data'] = pd.to_datetime(data['data'])
data = data.sort_values(by='data')

In [12]:
# Convert 'data' column to datetime format for easier filtering by year and month
data['data'] = pd.to_datetime(data['data'])

# Separate data for relevant years for reference
data_2019 = data[(data['ano'] == 2019)]
data_2022 = data[(data['ano'] == 2022) & (data['mes'] >= 2)]
data_2023_jan = data[(data['ano'] == 2023) & (data['mes'] == 1)]

# Prepare a copy of the data to apply the modifications
updated_data = data.copy()

# Function to calculate weighted mean based on specified weights
def weighted_mean(val_1, val_2, weight_1=2, weight_2=1):
    return (weight_1 * val_1 + weight_2 * val_2) / (weight_1 + weight_2)

# Apply the weighted mean adjustments for months in 2020 (April to December)
for month in range(4, 13):
    value_2019 = data_2019[data_2019['mes'] == month]['valor_arrecadacao'].values
    value_2022 = data_2022[data_2022['mes'] == month]['valor_arrecadacao'].values
    
    # Check if both values are available for calculation
    if len(value_2019) > 0 and len(value_2022) > 0:
        adjusted_value = weighted_mean(value_2019[0], value_2022[0])
        updated_data.loc[(updated_data['ano'] == 2020) & (updated_data['mes'] == month), 'valor_arrecadacao'] = adjusted_value

In [13]:
# Apply the weighted mean adjustments for all months in 2021
for month in range(1, 13):
    value_2022 = data_2022[data_2022['mes'] == month]['valor_arrecadacao'].values
    value_2019 = data_2019[data_2019['mes'] == month]['valor_arrecadacao'].values

    # Check if values are available from 2022 and 2023 for each month
    if month!=1:
        adjusted_value = weighted_mean(value_2022[0], value_2019[0])
        updated_data.loc[(updated_data['ano'] == 2021) & (updated_data['mes'] == month), 'valor_arrecadacao'] = adjusted_value
    if month==1:
        value_2023 = data_2023_jan[data_2023_jan['mes'] == month]['valor_arrecadacao'].values
        adjusted_value = weighted_mean(value_2023[0], value_2019[0])
        updated_data.loc[(updated_data['ano'] == 2021) & (updated_data['mes'] == month), 'valor_arrecadacao'] = adjusted_value

In [15]:
# Adjust for January 2022
value_2023_jan = data_2023_jan[data_2023_jan['mes'] == 1]['valor_arrecadacao'].values
value_2019_jan = data_2019[data_2019['mes'] == 1]['valor_arrecadacao'].values

adjusted_value_jan_2022 = weighted_mean(value_2023_jan[0], value_2019_jan[0], weight_1 = 3)
updated_data.loc[(updated_data['ano'] == 2022) & (updated_data['mes'] == 1), 'valor_arrecadacao'] = adjusted_value_jan_2022

In [16]:
# Save the modified data to a new file for review
updated_file_path = 'E:\\Dropbox\\transito\\receita\\receita_multas_ajustado.csv'
updated_data.to_csv(updated_file_path, index=False)

In [18]:
updated_data[updated_data['ano'] == 2021]

,data,ano,mes,valor_arrecadacao
85,2021-01-01,2021,1,2.135841e+07
28,2021-02-01,2021,2,2.056934e+07
69,2021-03-01,2021,3,2.224725e+07
7,2021-04-01,2021,4,1.742336e+07
74,2021-05-01,2021,5,2.422687e+07
81,2021-06-01,2021,6,1.985810e+07
35,2021-07-01,2021,7,2.395261e+07
78,2021-08-01,2021,8,2.025318e+07
47,2021-09-01,2021,9,2.130484e+07
51,2021-10-01,2021,10,1.763345e+07
